In [1]:
from openai import OpenAI
import os
import json
import time

In [2]:
input = {'story':{'text': 'In the realm of Eldoria, magic flows through the very air like an invisible river, influencing every aspect of life. Ancient kingdoms vie for power, their armies bolstered by sorcerers and mythical creatures, such as dragons and griffins. The land is scarred with the remnants of past wars between gods and mortals, and forgotten relics of divine power are said to rest in the depths of enchanted forests or the peaks of towering mountains. The balance between light and shadow is fragile, with the fate of the world hinging on a group of unlikely heroes destined to tip the scales.'}, 'player':{'name': '1111', 'description': '2222', 'level': 1, 'exp': 0, 'nextExp': 100, 'statPoints': 0, 'status': {'status': {'hp': 50, 'mp': 100, 'shield': 50, 'strength': 5, 'dexterity': 5, 'intelligence': 20, 'luck': 5, 'defense': 5, 'agility': 10, 'resistance': 10, 'toughness': 10, 'hp_regeneration': 0, 'mp_regeneration': 1}, 'origin_status': {'hp': 50, 'mp': 100, 'shield': 50, 'strength': 5, 'dexterity': 5, 'intelligence': 20, 'luck': 5, 'defense': 5, 'agility': 10, 'resistance': 10, 'toughness': 10, 'hp_regeneration': 0, 'mp_regeneration': 1}, 'added_status': {'hp': 0, 'mp': 0, 'shield': 0, 'strength': 0, 'dexterity': 0, 'intelligence': 0, 'luck': 0, 'defense': 0, 'agility': 0, 'resistance': 0, 'toughness': 0, 'hp_regeneration': 0, 'mp_regeneration': 0}}, 'attacks': [{'name': 'Frost Bolt', 'type': 'magic', 'defaultDamage': 80, 'coef': {'intelligence': 1}, 'count': 1, 'penetration': 5, 'accuracy': 85, 'cooldown': 3, 'statusEffect': {'type': 'Freeze', 'duration': 2, 'defaultValue': 0, 'coef': {'intelligence': 0.1}, 'accuracy': 40}, 'curCooldown': 0}, {'name': 'Arcane Missile', 'type': 'magic', 'defaultDamage': 15, 'coef': {'intelligence': 0.5}, 'count': 4, 'penetration': 5, 'accuracy': 90, 'cooldown': 2, 'curCooldown': 0}], 'defends': [{'name': 'Magic Shield', 'type': 'shield', 'defaultValue': 20, 'coef': {'intelligence': 1}, 'duration': 3, 'cooldown': 4, 'curCooldown': 0}], 'smites': [{'name': 'Mana Burst', 'type': 'damage', 'defaultValue': 30, 'coef': {'intelligence': 0.5}, 'cooldown': 5, 'curCooldown': 0}], 'inventory': {'items': [], 'equipments': {'helmet': None, 'armor': None, 'pants': None, 'shoes': None, 'gloves': None, 'rightHand': None, 'leftHand': None, 'ring1': None, 'ring2': None, 'earring1': None, 'earring2': None, 'necklace': None}}}}

In [5]:
message = [
    {
        "type": "text",
        "text": {"Worldview":input['story']['text']}
    },
    {
        "type": "text",
        "text": {"player":{
            "name": input['player']['name'],
            "description": input['player']['description'],
            "level": input['player']['level'],
            "status": input['player']['status'],
        }}
    }   
]

In [26]:
from openai import OpenAI
import os
import json
import time

from openai.types.beta.assistant_stream_event import (
    ThreadRunRequiresAction,
    ThreadMessageDelta,
    ThreadRunFailed,
    ThreadRunCancelling,
    ThreadRunCancelled,
    ThreadRunExpired,
    ThreadRunStepFailed,
    ThreadRunStepCancelled,
)

client = OpenAI(api_key = os.environ['OPENAI_API_KEY'])
STORY_ID = 'asst_QJULNvgbgB8cU1YgY5JzLAyr'
SKILL_ID = 'asst_JSej8B49OeeI7IwOUNp3dpKY'
ENEMY_ID = 'asst_ppQ8bsiqkd0w3G8N7tn7NUCb'
PENALTY_ID = 'asst_jjPhkdDYvzUWcto0DGh0x35r'
REWARD_ID = 'asst_YvLH1ztf6NC7qbLQGnlcUV4c'

def create_thread():
  return client.beta.threads.create()

def show_message(json_object):
  if 'Story' in json_object:
    for idx, choice in enumerate(json_object['Choices']):
      print(f'{idx+1}: {choice["text"]} ({choice["status"]}), Gold:({choice["gold"]}), ({choice["next_type"]})')
     

def run_thread(thread, user_message, assistant_id=STORY_ID):
  for idx in range(len(user_message)):
    user_message[idx]['text'] = str(user_message[idx]['text'])
  run = submit_message(user_message, thread, assistant_id)
  run = wait_run(run, thread)
  messages = get_message(thread)
  text = messages.data[0].content[0].text.value
  json_object=json.loads(text)

  show_message(json_object)
  time.sleep(0.5)
  return json_object


def run(thread, message):
  if 'Worldview' in message[0]['text']:
    # First message
    response = run_thread(thread, message, STORY_ID)
  elif 'next_type' in message[0]['text']:
    # Story selection
    if message[0]['text']['next_type'] == 'story':
      response = run_thread(thread, message, STORY_ID)
    elif message[0]['text']['next_type'] == 'combat':
      enemy = run_thread(thread, message, ENEMY_ID)
      enemy_info = str({'name':enemy['combat']['name'], 'description': enemy['combat']['description']})
      message = [
        {
          'type': 'text',
          'text': f'Make enemy skills for {enemy_info}'
        }
      ]
      skills = run_thread(thread, message, SKILL_ID)
      response = {'combat':dict(enemy['combat'], **skills)}
    elif message[0]['text']['next_type'] == 'reward':
      reward = run_thread(thread, message, REWARD_ID)
    elif message[0]['text']['next_type'] == 'penalty':
      penalty = run_thread(thread, message, PENALTY_ID)
    else:
      response = 'error'
      
  return response


def submit_message(user_message, thread, assistant_id=STORY_ID):
  for idx in range(len(message)):
    message[idx]['text'] = str(message[idx]['text'])
  client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message
  )
  run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
    stream=True
  )
  for event in run:
    if isinstance(event, ThreadMessageDelta):
      data = event.data.delta.content
      for d in data:
        for token in d:
          yield token


def wait_run(run, thread):
  while run.status == "queued" or run.status == "in_progress":
    # connect thread and assistant
    run = client.beta.threads.runs.retrieve(
      thread_id=thread.id,
      run_id=run.id
    )
    time.sleep(0.5)
  return run

def get_message(thread):
  return client.beta.threads.messages.list(thread_id=thread.id)

In [17]:
thread = create_thread()

In [29]:
for idx in range(len(message)):
    message[idx]['text'] = str(message[idx]['text'])
for i in submit_message(message, thread, STORY_ID):
    if i[0] == 'text':
        print(i[1].value, flush=True, end='')

{"story":"With a fierce determination, you harness your magic, drawing deep into your reservoir of power. In a blinding flash, you release a torrent of energy towards the Shade Wraith. The spell crackles through the air, striking the wraith with a burst of light that pierces through the darkness. The creature lets out an ear-splitting shriek as it is engulfed in a radiant explosion. For a moment, silence reigns in the grove, and then the wraith’s form begins to disintegrate, shadows fleeing from the light. As the echo of its last cry fades, the Shade Wraith disappears into nothingness, leaving only the crystal-clear Orb of Illumination on the pedestal before you, radiating warmth and magic. You’ve triumphed against darkness and secured the artifact—an omen of brighter days ahead.","choices":[{"text":"Pick up the Orb of Illumination and study its power.","status":{"hp":null,"mp":null,"strength":null,"dexterity":null,"intelligence":null,"luck":10,"defense":null,"agility":null,"resistance